In [87]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt

import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import log_loss
from keras import __version__ as keras_version

np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import pdb


from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn import linear_model,decomposition
from sklearn.feature_extraction import DictVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import StratifiedKFold
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp
import scipy

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression,SGDClassifier
from sklearn.linear_model import RidgeCV
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, ShuffleSplit

from scipy.sparse import coo_matrix, hstack ,vstack

import gc

print(gc.collect())

import xgboost as xgb
from keras import applications
import os

374


In [88]:
Listallimages=[]
ALB_path='inputdata/train/ALB'
BET_path='inputdata/train/BET'
DOL_path='inputdata/train/DOL'
LAG_path='inputdata/train/LAG'
SHARK_path='inputdata/train/SHARK'
YFT_path='inputdata/train/YFT'
NoF_path = 'inputdata/train/NoF'
OTHER_path = 'inputdata/train/OTHER'


images=os.listdir(ALB_path)
ALB_images=[ff for ff in images if '.jpg' in ff ]


images=os.listdir(BET_path)
BET_images=[ff for ff in images if '.jpg' in ff ]

images=os.listdir(DOL_path)
DOL_images=[ff for ff in images if '.jpg' in ff ]

images=os.listdir(LAG_path)
LAG_images=[ff for ff in images if '.jpg' in ff ]

images=os.listdir(SHARK_path)
SHARK_images=[ff for ff in images if '.jpg' in ff ]

images=os.listdir(YFT_path)
YFT_images=[ff for ff in images if '.jpg' in ff ]

images=os.listdir(NoF_path)
NoF_images=[ff for ff in images if '.jpg' in ff ]

images=os.listdir(OTHER_path)
OTHER_images=[ff for ff in images if '.jpg' in ff ]

Allimages={'ALB': ALB_images,
           'BET': BET_images,
           'DOL': DOL_images,
           'LAG': LAG_images,
           'SHARK': SHARK_images,
           'YFT': YFT_images,
           'NoF': NoF_images,
           'OTHER': OTHER_images}

In [89]:
# Getting the mean of red blue and green pixel over all images

In [90]:
mean_R = []
mean_G = []
mean_B = []

#loading images
for imageset in Allimages.keys():
    print imageset
    for i in Allimages[imageset]:
        img_path = 'inputdata/train/'+imageset+'/'+i
        img = image.load_img(img_path, target_size=(224, 224,3))
        
        #converting images to arrays
        x = image.img_to_array(img)
        
        #finding the mean image
        a = np.mean(x[:,:,0])
        mean_R = np.append(mean_R,a)
        b = np.mean(x[:,:,1])
        mean_G = np.append(mean_G,b)
        c = np.mean(x[:,:,2])
        mean_B = np.append(mean_B,c)

#Mean Image
I_R = np.mean(mean_R)
I_G = np.mean(mean_G)
I_B = np.mean(mean_B)
print (I_R,I_G,I_B)

YFT
SHARK
OTHER
DOL
NoF
LAG
ALB
BET
(96.403686592621185, 107.12191881841852, 99.894691938946167)


#  Resnet

In [95]:
Xtrain_resnet50=[]
Ytrain_resnet50=[]
count = 0
Y_top10 = {}

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')


#creating normalized image data set
for imageset in Allimages.keys():
    print imageset
    for i in Allimages[imageset]:
        img_path = 'inputdata/train/'+imageset+'/'+i
        img = image.load_img(img_path, target_size=(224, 224,3))
        
        x = image.img_to_array(img)
        x[:,:,0] = x[:,:,0] - I_R
        x[:,:,1] = x[:,:,1] - I_G
        x[:,:,2] = x[:,:,2] - I_B
    

        x = np.expand_dims(x, axis=0)
#       x = preprocess_input(x)

        preds = model.predict(x)
        Xtrain_resnet50.append(preds)
        Ytrain_resnet50.append(imageset)

        #decode predictions    
        Y_top10[i] = decode_predictions(preds,top=10)
        count += 1

#number of images processed
print count

YFT
SHARK
OTHER
DOL
NoF
LAG
ALB
BET
3777


In [107]:
Xtrain_resnet50=np.array(Xtrain_resnet50)
Ytrain_resnet50=np.array(Ytrain_resnet50)
Xtrain_resnet50.shape

Xtrain_resnet50=Xtrain_resnet50.reshape(3777,1000)
np.savez('ResnetFeatueExtact',Xtrain_resnet50=Xtrain_resnet50,Ytrain_resnet50=Ytrain_resnet50)

# VGG16

In [99]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
Xtrain_VGG16=[]
Ytrain_VGG16=[]
count = 0
Y_top10_VGG16 = {}

# from keras.applications.resnet50 import ResNet50
# from keras.preprocessing import image
# from keras.applications.resnet50 import preprocess_input, decode_predictions
# import numpy as np
# model = ResNet50(weights='imagenet')

model = VGG16(weights='imagenet', include_top=True)


#creating normalized image data set
for imageset in Allimages.keys():
    print imageset
    for i in Allimages[imageset]:
        img_path = 'inputdata/train/'+imageset+'/'+i
        img = image.load_img(img_path, target_size=(224, 224,3))
        
        x = image.img_to_array(img)
        x[:,:,0] = x[:,:,0] - I_R
        x[:,:,1] = x[:,:,1] - I_G
        x[:,:,2] = x[:,:,2] - I_B
    

        x = np.expand_dims(x, axis=0)
#       x = preprocess_input(x)

        preds = model.predict(x)
        Xtrain_VGG16.append(preds)
        Ytrain_VGG16.append(imageset)

        #decode predictions    
        Y_top10_VGG16[i] = decode_predictions(preds,top=10)
        count += 1

#number of images processed
print count

Xtrain_VGG16=np.array(Xtrain_VGG16)
Ytrain_VGG16=np.array(Ytrain_VGG16)
Xtrain_VGG16.shape
# img_path = 'inputdata/train/SHARK/img_00072.jpg'
# img = image.load_img(img_path, target_size=(224, 224,3))
# x = image.img_to_array(img)

# x = np.expand_dims(x, axis=0)
# # x = preprocess_input(x)

# features = model.predict(x)
# print features.shape

YFT
SHARK
OTHER
DOL
NoF
LAG
ALB
BET
3777


(3777, 1, 1000)

In [108]:
Xtrain_VGG16=np.array(Xtrain_VGG16)
Ytrain_VGG16=np.array(Ytrain_VGG16)
Xtrain_VGG16.shape

Xtrain_VGG16=Xtrain_VGG16.reshape(3777,1000)
np.savez('VGG16FeatueExtact',Xtrain_VGG16=Xtrain_VGG16,Ytrain_VGG16=Ytrain_VGG16)

# VGG19

In [100]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input, decode_predictions
# from keras.models import Model
import numpy as np
Xtrain_VGG19=[]
Ytrain_VGG19=[]
count = 0
Y_top10_VGG19 = {}

model = VGG19(weights='imagenet',include_top=True)
# model = Model(input=base_model.input, output=base_model.get_layer('block4_pool').output)

#creating normalized image data set
for imageset in Allimages.keys():
    print imageset
    for i in Allimages[imageset]:
        img_path = 'inputdata/train/'+imageset+'/'+i
        img = image.load_img(img_path, target_size=(224, 224,3))
        
        x = image.img_to_array(img)
        x[:,:,0] = x[:,:,0] - I_R
        x[:,:,1] = x[:,:,1] - I_G
        x[:,:,2] = x[:,:,2] - I_B
    

        x = np.expand_dims(x, axis=0)
#       x = preprocess_input(x)

        preds = model.predict(x)
        Xtrain_VGG19.append(preds)
        Ytrain_VGG19.append(imageset)

        #decode predictions    
        Y_top10_VGG19[i] = decode_predictions(preds,top=10)
        count += 1

#number of images processed
print count

Xtrain_VGG19=np.array(Xtrain_VGG19)
Ytrain_VGG19=np.array(Ytrain_VGG19)
Xtrain_VGG19.shape
# img_path = 'inputdata/train/SHARK/img_00072.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# block4_pool_features = model.predict(x)
# print block4_pool_features.shape



YFT
SHARK
OTHER
DOL
NoF
LAG
ALB
BET
3777


(3777, 1, 1000)

In [116]:
Xtrain_VGG19=np.array(Xtrain_VGG19)
Ytrain_VGG19=np.array(Ytrain_VGG19)
Xtrain_VGG19.shape

Xtrain_VGG19=Xtrain_VGG19.reshape(3777,1000)
np.savez('VGG19FeatueExtact',Xtrain_VGG19=Xtrain_VGG19,Ytrain_VGG19=Ytrain_VGG19)

# InceptionV3

In [112]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras.applications.inception_v3 import preprocess_input, decode_predictions
Xtrain_InceptionV3=[]
Ytrain_InceptionV3=[]
count = 0
Y_top10_InceptionV3 = {}

# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_data_format() == 'channels_last'

# model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)
model = InceptionV3(input_tensor=input_tensor,weights='imagenet', include_top=True)

#creating normalized image data set
for imageset in Allimages.keys():
    print imageset
    for i in Allimages[imageset]:
        img_path = 'inputdata/train/'+imageset+'/'+i
        img = image.load_img(img_path, target_size=(224, 224,3))
        
        x = image.img_to_array(img)
        x[:,:,0] = x[:,:,0] - I_R
        x[:,:,1] = x[:,:,1] - I_G
        x[:,:,2] = x[:,:,2] - I_B
    

        x = np.expand_dims(x, axis=0)
#       x = preprocess_input(x)

        preds = model.predict(x)
        Xtrain_InceptionV3.append(preds)
        Ytrain_InceptionV3.append(imageset)

        #decode predictions    
        Y_top10_InceptionV3[i] = decode_predictions(preds,top=10)
        count += 1

#number of images processed
print count

Xtrain_InceptionV3=np.array(Xtrain_InceptionV3)
Ytrain_InceptionV3=np.array(Ytrain_InceptionV3)
Xtrain_InceptionV3.shape
# img_path = 'inputdata/train/SHARK/img_00072.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# features = model.predict(x)
# print features.shape

YFT
SHARK
OTHER
DOL
NoF
LAG
ALB
BET
3777


(3777, 1, 1000)

In [117]:
Xtrain_InceptionV3=np.array(Xtrain_InceptionV3)
Ytrain_InceptionV3=np.array(Ytrain_InceptionV3)
Xtrain_InceptionV3.shape

Xtrain_InceptionV3=Xtrain_InceptionV3.reshape(3777,1000)
np.savez('InceptionV3FeatueExtact',Xtrain_InceptionV3 = Xtrain_InceptionV3,Ytrain_InceptionV3=Ytrain_InceptionV3)